In [11]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
#using Reel
using SparseArrays
using GridInterpolations
using DataStructures
using DataFrames
using CSV
using Distributed

In [33]:

include("atan2.jl")
include("obs_rel.jl")
include("polargrid_rel_qual2.jl")
rng = MersenneTwister(2);
#

In [13]:
# random walk

POS_0 = [6.0, 60.0]
CRS_0 = 90 # target's course
SPD_0 = 1 # 1 or 2
TGT_SPD = 1

#input is course in degrees and rng
#returns next course in degrees
function next_crs(crs,rng)
    if rand(rng) < .9
        return crs
    end
    crs = (crs + rand(rng,[-1,1])*30) % 360
    if crs < 0 crs += 360 end
    return crs
end

# state as tuple (x, y, crs, spd) of target (spd of o/s)
function f(state, control, rng)
    r, θ, crs, spd = state
    θ += control[1]
    spd = control[2]
    if θ < 0 θ += 360 end
    θ = θ % 360
    crs -= control[1]
    if crs < 0 crs += 360 end
    crs = crs % 360
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + TGT_SPD*cos(π/180*crs) - spd, y + 
        TGT_SPD*sin(π/180*crs)]
    crs = next_crs(crs,rng)
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = atan2(pos[1],pos[2])*180/π
    if θ < 0 θ += 360 end
    return (r, θ, crs, spd)::NTuple{4, Real}
end
#

f (generic function with 1 method)

In [14]:
function r(s)
    range = s[1]
    if range > 150 return -.2 end  # reward to not lose track of contact
    if range <= 10 return -100 end  # collision avoidance
    return .1  # being in "sweet spot" maximizes reward
end
#

r (generic function with 1 method)

In [ ]:
# this one is performing very well


In [15]:
#angles = [0, 30, 60, 90, 120, 150, 210, 240, 270, 300, 330]

statespace = thestates
actionspace = ((30,1), (0,1), (-30,1), (30, 2), (0,2), (-30,2))

action_index(a) = trunc(Int, 2*(a[1]/30+1) + a[2])
actions_ = ((-30,1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))



((-30, 1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))

In [16]:
# returns vector rather than Tuple, for particle filter
function f2(x, u, rng)
    temp = [i for i in f(x, u, rng)]
    return temp
end

f2 (generic function with 1 method)

In [17]:
totals = [0.0]
θ = zeros(length(grid),6);
e = sparse(zeros(length(grid),6))
#θ = [r(Tuple(ind2x(grid, j))) for j in 1:length(grid), i in 1:6];

In [21]:

N = 250
model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, N);
α = 0.2
γ = 0.9
ϵ = 0.1
x = [20, 60, 90, 1, 1];
λ = 0.9
b = ParticleCollection([[20, 60, 90, 1] for i in 1:N]);
counter = 0
epochsize = 200

0

In [ ]:
## Q-learning loop
#plots = []
#betas = Deque{Array}()
#β = zeros(length(grid),6);
# using updated wrapmap2
# length-200 epochs, epsilon-greedy

epochs = 15000


total = 0
rando = 0

last = 0
x = [rand(rng, 25:150), rand(rng,0:359), rand(rng,0:11)*30, 1, 1];     
b = ParticleCollection([x[1:4] for i in 1:N]);     
ξ = sparse(weighted_grid_2(b)/N)
uu = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
u = uu[1]
rando += uu[2]

for i in 1:(epochsize*epochs)
    counter += 1
    
    e[:,u] += ξ
    last = transpose(θ[:,u])*ξ
    
    # choose next action
    uu = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
    u = uu[1]
    rando += uu[2]
    a_star = uu[3]
    
    #observe new state and reward
    xp = f2(x, actions_[u], rng)
    y = h(xp, rng)
    b = update(pfilter, b, actions_[u], y)
    rew = r(Tuple(xp))
       
    ξ = sparse(weighted_grid_2(b)/N)
     
    total += rew
        
    #cur = (rew + γ * max2([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], rng))
    
    #argmax state-action pair, not realized state-action pair?
    cur = transpose(θ[:,a_star])*ξ
    
    δ = rew + γ * cur - last
    
    θ += α * δ * e
    #if uu[2] == 0
    e *= γ * λ
    #else
    #    e = sparse(zeros(length(grid),6))
    #end
    
        
    if (counter % epochsize == 0) || (xp[1] <= 10)
        push!(totals, total*500/epochsize)
        println("--------- CURRENT: ", round(total*500/epochsize, sigdigits=4), " AVG: ", round(mean(totals), sigdigits=4), " Epoch: ", 
            trunc(Int, counter/epochsize), " -----------")
        total = 0
        rando = 0
        xp = [rand(rng, 25:150), rand(rng,0:359), rand(rng,0:11)*30, rand(rng, [1,2]), 1];
        b = ParticleCollection([xp[1:4] for i in 1:N]);
        ξ = sparse(weighted_grid_2(b)/N)
        #ϵ = .75-counter/480000
        ## ϵ was .75
    end
    if counter % (epochsize*5) == 0
        thetaframe = DataFrame(θ)
        CSV.write("current47_1.csv", thetaframe)
    end
    
    x = xp
    
    
    #plotting
    #r_ = [row[1] for row in particles(b)]
    #theta = [row[2] for row in particles(b)]*π/180
    #x_theta = x[2]*π/180
    #x_r = x[1 ]
    
    #print(".")
    #plt = plot(proj=:polar, lims=(0,200), size=(1000,1000))
    #scatter!(theta, r_, markersize=1, label="particles")
    #scatter!([x_theta], [x_r], markersize=3, label="target")
          
    #push!(plots, plt)
    
end


--------- CURRENT: 35.0 AVG: 37.63 Epoch: 22244 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22245 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22246 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22247 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22248 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22249 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22250 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22251 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22252 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22253 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22254 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22255 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22256 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22257 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22258 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch: 22259 -----------
--------- CURRENT: 50.0 AVG: 37.64 Epoch

--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22508 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22509 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22510 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22511 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22512 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22513 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22514 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22515 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22516 -----------
--------- CURRENT: 50.0 AVG: 37.65 Epoch: 22517 -----------
--------- CURRENT: 50.0 AVG: 37.66 Epoch: 22518 -----------
--------- CURRENT: 50.0 AVG: 37.66 Epoch: 22519 -----------
--------- CURRENT: 50.0 AVG: 37.66 Epoch: 22520 -----------
--------- CURRENT: 50.0 AVG: 37.66 Epoch: 22521 -----------
--------- CURRENT: -242.5 AVG: 37.64 Epoch: 22521 -----------
--------- CURRENT: 42.25 AVG: 37.65 Epoch: 22522 -----------
--------- CURRENT: 50.0 AVG: 37.65 Ep

--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22638 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22639 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22640 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22641 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22642 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22643 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22644 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22645 -----------
--------- CURRENT: -10.75 AVG: 37.61 Epoch: 22646 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22647 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22648 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22649 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22650 -----------
--------- CURRENT: 50.0 AVG: 37.61 Epoch: 22651 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22652 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22653 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epo

--------- CURRENT: -21.25 AVG: 37.63 Epoch: 22772 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22773 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22774 -----------
--------- CURRENT: -230.3 AVG: 37.62 Epoch: 22774 -----------
--------- CURRENT: 30.0 AVG: 37.62 Epoch: 22775 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22776 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22777 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22778 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22779 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22780 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22781 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22782 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22783 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22784 -----------
--------- CURRENT: 30.5 AVG: 37.62 Epoch: 22785 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22786 -----------
--------- CURRENT: 50.0 AVG: 37.63 E

--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22903 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22904 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22905 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22906 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22907 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22908 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22909 -----------
--------- CURRENT: -94.0 AVG: 37.62 Epoch: 22910 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22911 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22912 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22913 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22914 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22915 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22916 -----------
--------- CURRENT: 50.0 AVG: 37.62 Epoch: 22917 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoch: 22918 -----------
--------- CURRENT: 50.0 AVG: 37.63 Epoc

...

--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29091 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29092 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29093 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29094 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29095 -----------
--------- CURRENT: 23.0 AVG: 38.79 Epoch: 29096 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29097 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29098 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29099 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29100 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29101 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29102 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29103 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29104 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29105 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch: 29106 -----------
--------- CURRENT: 50.0 AVG: 38.79 Epoch

--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29226 -----------
--------- CURRENT: -238.5 AVG: 38.8 Epoch: 29226 -----------
--------- CURRENT: 38.25 AVG: 38.8 Epoch: 29227 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29228 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29229 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29230 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29231 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29232 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29233 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29234 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29235 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29236 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29237 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29238 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29239 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29240 -----------
--------- CURRENT: 50.0 AVG: 38.8 Epoch: 29241 -----

--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29359 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29360 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29361 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29362 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29363 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29364 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29365 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29366 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29367 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29368 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29369 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29370 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29371 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29372 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29373 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch: 29374 -----------
--------- CURRENT: 50.0 AVG: 38.81 Epoch

--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29496 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29497 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29498 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29499 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29500 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29501 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29502 -----------
--------- CURRENT: 50.0 AVG: 38.85 Epoch: 29503 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29504 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29505 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29506 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29507 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29508 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29509 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29510 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29511 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch

--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29629 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29630 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29631 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29632 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29633 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29634 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29635 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29636 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29637 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29638 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29639 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29640 -----------
--------- CURRENT: 50.0 AVG: 38.86 Epoch: 29641 -----------
--------- CURRENT: 50.0 AVG: 38.87 Epoch: 29642 -----------
--------- CURRENT: 50.0 AVG: 38.87 Epoch: 29643 -----------
--------- CURRENT: 50.0 AVG: 38.87 Epoch: 29644 -----------
--------- CURRENT: 50.0 AVG: 38.87 Epoch

In [30]:
#transpose(θ[:,2])*ξ
epochsize

200

In [ ]:
xx = []
yy = []
yyy = []
for (i, t) in enumerate(totals)
    push!(xx, i)
    push!(yy, t)
    push!(yyy, mean(yy))
end

plot(xx, yyy, ylims = (-20,50))

In [10]:
#θ = convert(Matrix, CSV.read("good.csv"))


28080×6 Array{Float64,2}:
  0.106108  0.0  -0.0829134  0.0   0.0743955  0.0
 -0.218553  0.0  -1.1224     0.0  -0.0545906  0.0
  0.326478  0.0  -1.73706    0.0  -0.277184   0.0
  0.908624  0.0  -1.52726    0.0   0.388403   0.0
  0.697881  0.0   0.670758   0.0   0.861652   0.0
 -0.10886   0.0  -1.74887    0.0   0.265702   0.0
  0.375173  0.0   1.65491    0.0   0.651283   0.0
  1.15377   0.0   6.51786    0.0   1.21614    0.0
  1.19869   0.0   5.65495    0.0  -0.0774921  0.0
  3.01445   0.0   9.6408     0.0  -0.425889   0.0
  4.68788   0.0   7.16558    0.0   1.04943    0.0
  4.66821   0.0   3.13691    0.0   1.33348    0.0
  6.16342   0.0   2.09777    0.0   1.28498    0.0
  ⋮                                           ⋮  
  0.0       0.0   0.0        0.0   0.0        0.0
  0.0       0.0   0.0        0.0   0.0        0.0
  0.0       0.0   0.0        0.0   0.0        0.0
  0.0       0.0   0.0        0.0   0.0        0.0
  0.0       0.0   0.0        0.0   0.0        0.0
  0.0       0.0   0.0   

In [18]:
θ = convert(Matrix, thetaframe)

28080×6 Array{Float64,2}:
 -1.77704    0.0  -3.91139    0.0  -1.6716    0.0
  0.104306   0.0  -0.277353   0.0   0.29317   0.0
  0.0992801  0.0   2.52568    0.0   1.46481   0.0
  0.892081   0.0   2.81689    0.0   1.36093   0.0
  0.633067   0.0   0.479396   0.0   0.563874  0.0
  1.14868    0.0   0.109032   0.0   0.678002  0.0
  0.651611   0.0  -0.0357058  0.0   0.760664  0.0
  1.14689    0.0   1.85016    0.0   0.111559  0.0
  1.16463    0.0   0.581399   0.0   0.865836  0.0
  1.1367     0.0   1.04617    0.0   1.66769   0.0
  0.345221   0.0   0.929168   0.0   0.788199  0.0
  0.681581   0.0   1.02612    0.0   0.704559  0.0
  0.978036   0.0   0.321273   0.0   1.28954   0.0
  ⋮                                           ⋮  
  0.0        0.0   0.0        0.0   0.0       0.0
  0.0        0.0   0.0        0.0   0.0       0.0
  0.0        0.0   0.0        0.0   0.0       0.0
  0.0        0.0   0.0        0.0   0.0       0.0
  0.0        0.0   0.0        0.0   0.0       0.0
  0.0        0.0   0.0  

In [ ]:
thetaframe = DataFrame(θ)
CSV.write("0.471_final.csv", thetaframe)

In [51]:
b = ParticleCollection([[20, 60, 90, 1] for i in 1:N]);
ξ = weighted_grid_2(b)/N
var(ξ)


3.5612535612535624e-5